### transfer learning example
The problem with many real-world tasks is that it is difficult to find large and high-quality datasets for them, and the process of training from scratch takes a lot of time and computational resources, making it quite expensive overall. Moreover, designing and constructing an efficient model is usually a serious scientific challenge.

This is where transfer learning comes in — a concept where we use pre-trained models to solve similar tasks. In general, transfer learning differs significantly from standard machine learning tasks and involves various techniques to adjust existing models. For example, a popular practice is to use convolutional networks that have been trained on the classic ImageNet dataset (http://www.image-net.org), and tweak them slightly to suit your specific task.

In [17]:
# Setup necessary imports and environment
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  # Interactive mode

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define data transformations
data_transforms = {
    'Training': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load fruits dataset (https://www.kaggle.com/datasets/moltean/fruits)
# Actually we are using only Watermelon and Mango
data_dir = 'data/fruits-360_dataset/fruits-360'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['Training', 'Test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['Training', 'Test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['Training', 'Test']}
class_names = image_datasets['Training'].classes
print("Classes:", class_names)

# Define model, loss function, optimizer, and scheduler
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Train the model (you can use the same training loop structure from the first notebook)
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['Training', 'Test']:
            if phase == 'Training':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'Training'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'Training':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'Training':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'Test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Execute training
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Classes: ['Mango', 'Watermelon']
Epoch 0/24
----------
Training Loss: 0.2690 Acc: 0.8953
Test Loss: 0.0001 Acc: 1.0000

Epoch 1/24
----------
Training Loss: 0.1771 Acc: 0.9461
Test Loss: 0.0001 Acc: 1.0000

Epoch 2/24
----------
Training Loss: 0.1281 Acc: 0.9565
Test Loss: 0.0000 Acc: 1.0000

Epoch 3/24
----------
Training Loss: 0.2864 Acc: 0.9192
Test Loss: 0.0000 Acc: 1.0000

Epoch 4/24
----------
Training Loss: 0.2816 Acc: 0.9192
Test Loss: 0.0000 Acc: 1.0000

Epoch 5/24
----------
Training Loss: 0.1784 Acc: 0.9523
Test Loss: 0.0000 Acc: 1.0000

Epoch 6/24
----------
Training Loss: 0.0685 Acc: 0.9731
Test Loss: 0.0001 Acc: 1.0000

Epoch 7/24
----------
Training Loss: 0.0437 Acc: 0.9824
Test Loss: 0.0000 Acc: 1.0000

Epoch 8/24
----------
Training Loss: 0.0262 Acc: 0.9917
Test Loss: 0.0000 Acc: 1.0000

Epoch 9/24
----------
Training Loss: 0.0217 Acc: 0.9948
Test Loss: 0.0000 Acc: 1.0000

Epoch 10/24
----------
Training Loss: 0.0454 Acc: 0.9845
Test Loss: 0.0000 Acc: 1.0000

Epoch 11/